In [65]:
!pip install -qU openai pinecone-client datasets pinecone-client pyarrow

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import openai, pinecone, os
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
# get API key from top-right dropdown on OpenAI website
openai.api_key = os.environ.get('API_KEY')

In [67]:
pinecone.init(
	api_key='90e334e8-7457-4a5c-a336-0d730e3a6606',
	environment='us-west4-gcp-free'
)
index = pinecone.Index('hadid')

In [68]:
embed_model = "text-embedding-ada-002"

In [69]:
################
#This loads the embedding and puts it in pinecone

In [70]:
file_path = 'E:\MyWorks\ReligionBotTry2\Embedding\hadith_correct_order.parquet'
data = pd.read_parquet(file_path)

In [71]:
data[:3]

,Volume,Book,Number,Narrator,Text,Embedding
0,1,BOOK 1. REVELATION (1-6),1,'Umar bin Al-Khattab,"I heard Allah's Apostle saying, ""The reward of...","[-0.013725720345973969, -0.03416065499186516, ..."
1,1,BOOK 1. REVELATION (1-6),2,'Aisha,(the mother of the faithful believers) Al-Hari...,"[-0.016002636402845383, -0.021992305293679237,..."
2,1,BOOK 1. REVELATION (1-6),3,'Aisha,(the mother of the faithful believers) The com...,"[-0.013475515879690647, -0.029280133545398712,..."


In [72]:
# vectors_to_upsert = [("0", data["Embedding"][0].tolist(), {"text": data["Text"][0]})]
# index.upsert(vectors_to_upsert)

In [73]:
import itertools

# Define the chunking function
def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

# Your existing data processing
vectors_to_upsert = [(str(index), row["Embedding"].tolist(),{"text":row["Text"]}) for index, row in data.iterrows()]

# Perform the upsert in chunks
for ids_vectors_chunk in chunks(vectors_to_upsert, batch_size=100):
    index.upsert(vectors=ids_vectors_chunk)

In [74]:
######
#api call w/ context now

In [75]:
text = "who is the creator of world"#$input("What troubles you?")
res = openai.Embedding.create(input=[text], engine=embed_model)
xq = res['data'][0]['embedding']
ok = index.query(xq, top_k=3,include_values=True, include_metadata=True)

In [76]:
ok['matches'][0]['metadata']['text']

'I heard Muhammad saying, "Whoever makes a picture in this world will be asked to put life into it on the Day of Resurrection, but he will not be able to do so."'

In [77]:
ok

{'matches': [{'id': '5534',
              'metadata': {'text': 'I heard Muhammad saying, "Whoever makes a '
                                   'picture in this world will be asked to put '
                                   'life into it on the Day of Resurrection, '
                                   'but he will not be able to do so."'},
              'score': 0.761447549,
              'values': [-0.025677368,
                         -0.0125718731,
                         -0.00156671961,
                         0.00683543971,
                         -0.0162500516,
                         0.000708716281,
                         -0.0450275056,
                         -0.007940799,
                         -0.00895086862,
                         -0.00877934694,
                         0.0328812599,
                         0.0303910263,
                         0.0152336285,
                         -0.00775021967,
                         0.0114792194,
                       

In [78]:
text = input("What troubles you?")
res = openai.Embedding.create(input=[text], engine=embed_model)
xq = res['data'][0]['embedding']
ok = index.query(xq, top_k=3,include_values=True, include_metadata=True)

contexts = []
for match in ok['matches']:
    #id_num = int(match['id'])
    contexts.append(match['metadata']['text'])

context = " ".join(contexts)
completion = openai.ChatCompletion.create(
  model="gpt-4-32k",
  messages=[
    {"role": "system", "content": f"You have been given the context of Islamic Book of hadis. User will ask you questions. Based on the given context, give a straight intelligent 2 sentence answer after understanding the context {context}. Give straight answer"},
    {"role": "user", "content": text}
  ]
)

print(completion.choices[0].message["content"] + '\n' + "Consider these verses in Hadis " + "\n" + "- " + contexts[0] + "\n" + "- " +contexts[1] + "\n" + "- " + contexts[2])

The context provided does not provide information on who is the leader of the free world. It mainly discusses the leadership of Usama bin Zaid as appointed by Allah's Apostle.
Consider these verses in Hadis 
- Allah's Apostle sent an army detachment and made Usama bin Zaid its commander. Some people criticized (spoke badly of) Usama's leadership. So Allah's Apostle got up saying, "If you people are criticizing Usama's leadership, you have already criticized the leadership of his father before. But Wa-aimullah (i.e., By Allah), he (i.e. Zaid) deserved the leadership, and he was one of the most beloved persons to me; and now this (his son Usama) is one of the dearest persons to me after him." (See Hadith No. 745, Vol. 5)
- The Prophet sent an army under the command of Usama bin Zaid. When some people criticized his leadership, the Prophet said, "If you are criticizing Usama's leadership, you used to criticize his father's leadership before. By Allah! He was worthy of leadership and was o